In [14]:
import json
import glob

In [15]:
import os
os.getcwd()

'/Users/matthewdeverna/Documents/Projects/top_fb_domains_trackers/notebooks'

In [16]:
glob.glob('../data/*')

['../data/account_info_2022-08-17.json',
 '../data/citizenbrowser_tweets_flattened_2022-08-17.jsonl',
 '../data/account_ids.txt',
 '../data/account_data_clean_2022-08-17.csv',
 '../data/citizenbrowser_tweets_2022-08-17.json',
 '../data/fb_top10_tweets_2022-08-17.json',
 '../data/fb_top10_tweets_flattened_2022-08-17.jsonl']

In [17]:
tweet_fp = '../data/citizenbrowser_tweets_flattened_2022-08-17.jsonl'

tweets = []

with open(tweet_fp, 'r') as f:
    for line in f:
        tweet = json.loads(line)
        tweets.append(tweet)

In [18]:
len(tweets)

762

In [88]:
def is_tweet_i_want(tweet_text):
    """
    Check that the tweet contains all top ten positions.
    
    text_tweet (str) : the tweet text
    
    output (bool) : whether it is a tweet we want or not
    """
    topten = ['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.']
    return any(rank in tweet_text for rank in topten)

In [77]:
def get_top_ten(tweet_text):
    ranks = ['1.', '2.', '3.', '4.', '5.', '6.', '7.', '8.', '9.', '10.']
    
    topten = []
    
    for obj in tweet_text.split('\n'):
        
        for rank in ranks:
            if obj.startswith(rank):
                topten.append( obj.replace(f"{rank} ", '') )
    
    return topten

In [89]:
from collections import defaultdict

all_data = defaultdict(list)

for tweet in tweets:
    tweet_text = tweet['text']
    
    if not is_tweet_i_want(tweet_text):
        print(tweet_text)
        print(tweet['id'])
        continue

    created_dt = dateutil.parser.isoparse(tweet['created_at'])
    date_key = created_dt.strftime("%Y_%m")
    
    try:
#         print("parsing top ten...")
        top_ten = get_top_ten(tweet_text)
    except:
        print("Ran into a problem parsing the top ten. Skipping the below tweet...")
        print("-"*50)
        print(tweet_text)
        print("-"*50)
    
    all_data[date_key].extend(top_ten)

ESPN FC
1558121200357892097
allkpop
1557758812525830145
kpop
1551235823273656320
C Sports Bay Area / California
1539277014531727361
al
1527318221145051138
s
1520071139736928256
CNN
1516446578894065677
ို
1509561210894798851
. Ben Shapiro
1482744509033336841
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1480570179323142152
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1474409585083662337
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1474047198044340226
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1471510486931755019
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1470060929916325889
iams
1466437052367527940
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1458102127449067541
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1452303922027192328
Note: posts by UNICEF are boosted by a Facebook Covid-19 info panel.
1451579144119201798
Note:

In [87]:
all_data.keys()

dict_keys(['2022_08', '2022_07', '2022_06', '2022_05', '2022_04', '2022_03', '2022_02', '2022_01', '2021_12', '2021_11', '2021_10', '2021_09', '2021_08', '2021_07', '2021_06', '2021_05', '2021_04', '2021_03', '2021_02', '2021_01', '2020_12', '2020_11', '2020_10', '2020_09', '2020_08', '2020_07'])

In [91]:
from collections import Counter

In [94]:
total_counter = Counter()

for key, entities in all_data.items():
    
    for ent in entities:
        total_counter[ent] += 1

In [97]:
import pandas as pd

In [102]:
total_counts_frame = pd.DataFrame.from_dict(total_counter, orient='index').reset_index()

In [104]:
total_counts_frame = total_counts_frame.rename(columns={'index':'entity_name', 0:'num_days'})

In [108]:
import altair as alt

In [111]:
total_counts_frame = total_counts_frame.sort_values('num_days', ascending=False).reset_index(drop=True)

In [112]:
total_counts_frame

,entity_name,num_days
0,Ben Shapiro,991
1,Dan Bongino,633
2,Fox News,464
3,Franklin Graham,265
4,CNN,222
...,...,...
638,News Channel 3 WWMT TV,1
639,Taunton Daily Gazette,1
640,Nelly,1
641,CNBC Make It,1


In [116]:
all_data.keys()

dict_keys(['2022_08', '2022_07', '2022_06', '2022_05', '2022_04', '2022_03', '2022_02', '2022_01', '2021_12', '2021_11', '2021_10', '2021_09', '2021_08', '2021_07', '2021_06', '2021_05', '2021_04', '2021_03', '2021_02', '2021_01', '2020_12', '2020_11', '2020_10', '2020_09', '2020_08', '2020_07'])

In [134]:
bars = alt.Chart(
    total_counts_frame.head(50),
    title = "Appearances in @FacebooksTop10 since July 2020"
).mark_bar().encode(
    x = alt.X("num_days:Q", title = 'Number of days in top ten'),
    y = alt.Y('entity_name:N', sort='-x', title = None)
)

final = bars + bars.mark_text(dx=15,fontSize=15).encode(
    x = alt.X("num_days:Q"),
    y = alt.Y('entity_name:N', sort='-x'),
    text = alt.Text("num_days")
).properties(width=400)


final.configure_axis(
    titleFontSize=15,
    labelFontSize=15
).configure_title(
    fontSize = 15
)

alt.LayerChart(...)

In [107]:
total_counts_frame.sort_values('num_days', ascending=False).head(50)

,entity_name,num_days
2,Ben Shapiro,991
10,Dan Bongino,633
4,Fox News,464
25,Franklin Graham,265
14,CNN,222
20,Occupy Democrats,217
0,NPR,202
5,Breitbart,178
12,allkpop,103
27,The New York Times,80


In [93]:
Counter(all_data['2021_09'])

Counter({'WTAE-TV Pittsburgh': 1,
         'NPR': 9,
         'Dan Bongino': 8,
         'Love Meow': 2,
         'AP': 2,
         'Inspire Uplift': 4,
         'Koreaboo': 3,
         'ABC30 Action News': 1,
         'Washington Post': 1,
         'Insider': 1,
         'CNN': 8,
         'Occupy Democrats': 1,
         'The New York Times': 4,
         'The Onion': 1,
         'Peachy Sunday': 5,
         'HuffPost': 2,
         'Reuters': 1,
         'Yahoo Life': 1,
         'The Real Mike Epps': 6,
         'Ben Shapiro': 20,
         'Breitbart': 8,
         'Soompi': 1,
         'allkpop': 7,
         'TMZ': 1,
         'NBC News': 2,
         'News 4 Tucson - KVOA': 1,
         'The Hollywood Reporter': 1,
         'The Dodo': 4,
         'Nick Adams': 4,
         '11Alive': 1,
         'IndieWire': 1,
         'ForAmerica': 3,
         'Animal Rescue Stories': 1,
         'Deadline Hollywood': 1,
         'Fox News': 12,
         'VOA Burmese News': 2,
         'Right Side Br

In [67]:
print(tweet_text)

The top-performing link posts by U.S. Facebook pages in the last 24 hours are from:

1. NPR
2. Todo Cuba
3. Ben Shapiro
4. A Woman’s Soul
5. Fox News
6. Breitbart
7. The Life of Animals
8. Fox News
9. Charlie Kirk
10. People


In [76]:
get_top_ten(tweet_text)

['NPR',
 'Todo Cuba',
 'Ben Shapiro',
 'A Woman’s Soul',
 'Fox News',
 'Breitbart',
 'The Life of Animals',
 'Fox News',
 'Charlie Kirk',
 'People']

In [74]:
tweet_text.split('\n')[3].replace("2. ", '')

'Todo Cuba'

In [20]:
tweets[0].keys()

dict_keys(['context_annotations', 'reply_settings', 'public_metrics', 'entities', 'author_id', 'lang', 'created_at', 'id', 'source', 'text', 'possibly_sensitive', 'conversation_id', 'author', '__twarc'])

In [44]:
import dateutil


In [50]:
dtobj = dateutil.parser.isoparse(tweets[6]['created_at'])

In [55]:
dtobj.

11

In [49]:
tweets[5].keys()

dict_keys(['context_annotations', 'reply_settings', 'public_metrics', 'entities', 'author_id', 'lang', 'created_at', 'id', 'source', 'text', 'possibly_sensitive', 'conversation_id', 'author', '__twarc'])

In [40]:


is_tweet_i_want(tweets[5]['text'])

True

In [ ]:
from osometweet.utils import 